In [1]:
import os
import sys
import time
import tempfile
import logging
import argparse
import subprocess
from lib import ApytramNeeds
from lib import BlastPlus
from lib import Trinity
from lib import Aligner


start_time = time.time()

In [2]:
### Option parsing
parser = argparse.ArgumentParser(prog = "apytram.py",
                                 description='''
    Run apytram on a fastq file to retrieve
    homologous sequences of bait sequences.''')

requiredOptions = parser.add_argument_group('required arguments')
requiredOptions.add_argument('-d', '--database', nargs='?', type=str,
                             help='Database preffix name', required=True)

parser.add_argument('--version', action='version', version='%(prog)s 1.0')
parser.add_argument('-log', nargs='?', type=str, default="apytram.log")
parser.add_argument('-t', '--tmp',  type=str,
                    help = "Directory to stock intermediary files for the apytram run. (default: a directory in /tmp which will be removed at the end)",
                    default = "" )
parser.add_argument('--threads',  type=int,
                    help = "Available threads. (Default 1)",
                    default = 1 )



parser.add_argument('-fa', '--fasta',  type=str,
                   help = "Fasta formatted RNA-seq data to build the database of reads")
parser.add_argument('-fq', '--fastq',  type=str,
                   help = "Fastq formatted RNA-seq data to build the database of reads")
parser.add_argument('-out', '--output',  type=str, default = "./")


parser.add_argument('-q', '--query',  type=str,
                    help = "Fasta file (nt) with bait sequence for the apytram run." )
parser.add_argument('-i', '--iteration_max',  type=int,
                    help = "Maximum number of iteration. (Default 5)",
                    default = 5 )
parser.add_argument('-e', '--evalue',  type=float,
                    help = "Evalue. (Default 1e-3)",
                    default = 1e-3 )

parser.add_argument('-id', '--min_id',  type=int,
                    help = "Minimum identity percentage with a query to keep a sequence  (Default 0)",
                    default = 0 )
parser.add_argument('-l', '--min_len',  type=int,
                    help = "Minimum length to keep a sequence  (Default 200)",
                    default = 200 )





#args = parser.parse_args()
args = parser.parse_args(''' -d example_exec/db/examplefq -out Out_test -fq example/example_db.fastq
                             -q example/ref_gene.fasta -t example_exec/tmp
                             -l 200
                             -i 10'''.split())

# Arguments reading
MaxIteration = args.iteration_max
Threads = args.threads
Evalue = args.evalue
MinIdentityPercentage = args.min_id
MinLength = args.min_len

In [3]:
### Set up the logger
LogFile = args.log
# create logger with 'spam_application'
logger = logging.getLogger('apytram')
logger.setLevel(logging.DEBUG)
# create file handler which logs even debug messages
fh = logging.FileHandler(LogFile)
fh.setLevel(logging.DEBUG)
# create console handler with a higher log level
ch = logging.StreamHandler()
ch.setLevel(logging.ERROR)
# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
ch.setFormatter(formatter)
# add the handlers to the logger
logger.addHandler(fh)
logger.addHandler(ch)

In [4]:
### Set up the working directory
if args.tmp:
    if os.path.isdir(args.tmp):
        logger.info("The temporary directory %s exists" %(args.tmp) )
    else:
        logger.info("The temporary directory %s does not exist, it will be created" % (args.tmp))
        os.makedirs(args.tmp)
    TmpDirName = args.tmp
else:
    TmpDirName = tempfile.mkdtemp()

# Remove only empty directory
#os.rmdir(TmpDirName)

INFO:apytram:The temporary directory example_exec/tmp exists


In [5]:
### Set up the output directory
if args.output:
    if os.path.isdir(args.output):
        logger.info("The output directory %s exists" %(args.output) )
    else:
        logger.info("The temporary directory %s does not exist, it will be created" % (args.output))
        os.makedirs(args.output)
    OutDirName = args.output
else:
    logger.error("The output directory must be defined")
    sys.exit(1)

INFO:apytram:The output directory Out_test exists


In [6]:
### Check that there is a database else built it
DatabaseName = args.database
if not os.path.isfile(DatabaseName+".nhr"):
    logger.info(DatabaseName+".nhr does not exist")
    #Build blast formated database from a fasta file
    if args.fastq or args.fasta:
        if args.fastq:
            if not os.path.isfile(args.fastq):
                logger.error("The fastq file (-fq) does not exist.")
                sys.exit(1)
            else:
                # Format the fastq file in fasta
                InputFasta = TmpDirName + "/" + os.path.basename(args.fastq) + ".fasta"
                ExitCode = ApytramNeeds.fastq2fasta(args.fastq,InputFasta)
        elif args.fasta:
            InputFasta = args.fasta
            
        if not os.path.isfile(InputFasta):
            logger.error("The fasta file (-fa) does not exist.")
            sys.exit(1)
        if os.path.isdir(os.path.dirname(DatabaseName)):
            logger.info("Database directory exists")
        else:
            logger.info("Database directory does not exist, we create it")
            os.makedirs(os.path.dirname(DatabaseName))
        # database building
        logger.info(DatabaseName + " database building")
        MakeblastdbProcess = BlastPlus.Makeblastdb(InputFasta,DatabaseName)
        ExitCode = MakeblastdbProcess.launch()
    else :
        logger.error("The database is not formatted ! A fasta file (-fa) or a fastq file (-fq) is required !")
        sys.exit(1)
if not os.path.isfile(DatabaseName+".nhr"):
    logger.error("Problem in the database building")
    logger.info(DatabaseName+".nhr does not exist")
    sys.exit(1)
else:
    logger.info(DatabaseName+".nhr exists")

INFO:apytram:example_exec/db/examplefq.nhr exists


In [7]:
### If there is a query continue, else stop
if not args.query:
    logger.info("There is no query (-q), apytram have finished.")
    quit()
elif not os.path.isfile(args.query):
    logger.error(args.query+" (-q) is not a file.")
    sys.exit(1)
else:
    queryFile = args.query
    QueryDatabaseName = TmpDirName + "/" + os.path.basename(queryFile).split(".")[0]
    BlastdbcmdProcess = BlastPlus.Makeblastdb(queryFile,QueryDatabaseName)
    ExitCode = BlastdbcmdProcess.launch()
    logger.info("apytram will run with \"%s\" as reads database and \"%s\" as bait sequences" %(DatabaseName,queryFile))
    

INFO:apytram:apytram will run with "example_exec/db/examplefq" as reads database and "example/ref_gene.fasta" as bait sequences


In [8]:
### Make iterations
# Initialisation
i = 0
Stop = False
LastIterStat = [0, 0]
BaitSequences = queryFile

logger.info("Iterations begin")
while (i < MaxIteration) and (Stop == False):
    i+=1
    start_iter = time.time()
    logger.info("Iteration %d/%d" %(i,MaxIteration))
    # Blast bait seqeunce on database of reads
    logger.info("Blast bait sequences on reads database")
    ReadNamesFile = TmpDirName + "/ReadNames.%d.txt" % (i)
    BlastnProcess = BlastPlus.Blast("blastn", DatabaseName, BaitSequences)
    BlastnProcess.Evalue = Evalue
    BlastnProcess.Threads = Threads
    BlastnProcess.OutFormat = "6 sacc"
    # Write read names in ReadNamesFile
    ExitCode = BlastnProcess.launch(ReadNamesFile)
    BlastnProcess.OutFormat = "6"
    ExitCode = BlastnProcess.launch(ReadNamesFile+".completetable")
    # Get paired reads names
    ExitCode = ApytramNeeds.add_paired_read_names(ReadNamesFile)
    # Retrieve sequences
    logger.info("Retrieve sequences")
    ReadFasta = TmpDirName + "/Reads.%d.fasta" % (i)
    BlastdbcmdProcess = BlastPlus.Blastdbcmd(DatabaseName, ReadNamesFile, ReadFasta)
    BlastdbcmdProcess.launch()
    # Launch Trinity
    start_trinity_time = time.time()
    logger.info("Launch Trinity")
    TrinityFasta = TmpDirName + "/Trinity_iter_%d" % (i)
    TrinityProcess = Trinity.Trinity(ReadFasta,TrinityFasta)
    # Keep only contig with a length superior to MinLength
    TrinityProcess.MinLength = MinLength
    # Use the  --full_cleanup Trinity option to keep only the contig file
    ExitCode = 0
    #ExitCode = TrinityProcess.launch("full_cleanup")
    TrinityFasta = TrinityFasta + ".Trinity.fasta"
    print("trinity --- %s seconds ---" % (time.time() - start_trinity_time))
    if ExitCode != 0: # Trinity found nothing
        logger.info("Trinity found nothing (ExitCode: %d)" %exitCode)
        Stop = True
    else:
        # Filter Trinity contigs to keep only homologous sequences of the reference genes
        logger.info("Compare Trinity results with query sequences")
        
        # Exonerate method
        TrinityExonerate = TmpDirName + "/Trinity_iter_%d.exonerate" % (i)
        start_exo_time = time.time()
        ExonerateProcess = Aligner.Exonerate(queryFile,TrinityFasta)
        # We want to keep only the best hit for each Trinity sequences
        ExonerateProcess.Bestn = 1
        ExonerateProcess.Model = "cdna2genome"
        # We customize our output format
        ExonerateProcess.Ryo = "%ti\t%qi\t%ql\t%tal\t%tl\t%tab\t%tae\t%s\t%pi\t%qab\t%qae\n"
        ExonerateResult = ExonerateProcess.get_output()
        # We write the result in a file
        ExonerateFile = open(TrinityExonerate,"w")
        ExonerateFile.write(ExonerateResult)
        ExonerateFile.close()
        # Keep only sequence with a identity percentage > MinIdentitypercentage on the whole hit
        FilteredSequenceNames, BestScoreNames, ExonerateResultsDict = ApytramNeeds.parse_exonerate_results(ExonerateResult, MinIdentityPercentage)
        print("exonerate --- %s seconds ---" % (time.time() - start_exo_time))
        
        # Blast method
        start_bla_time = time.time()
        TrinityBlast = TmpDirName + "/Trinity_iter_%d.blast" % (i)
        BlastnProcess = BlastPlus.Blast("blastn", QueryDatabaseName, TrinityFasta)
        BlastnProcess.OutFormat = "6"
        BlastnProcess.Evalue = 1e-8
        BlastnProcess.perc_identity = 80
        BlastnProcess.launch(TrinityBlast)
        print("blast --- %s seconds ---" % (time.time() - start_bla_time))
        
        # Filter hit
        logger.info("Filter sequence with a identity percentage superior to %d" %(MinIdentityPercentage)) 
        FileteredTrinityFasta = TmpDirName + "/Trinity_iter_%d.filtered.fasta" % (i)
        ExitCode = ApytramNeeds.filter_fasta(TrinityFasta,FilteredSequenceNames,FileteredTrinityFasta)
        
        # Validated sequences become bait sequences
        BaitSequences = FileteredTrinityFasta
        
        # Compare to the previous iteration
        start_mafft_time = time.time()
        logger.info("Compare results with the previous iteration")
        MafftProcess = Aligner.Mafft(queryFile)
        MafftProcess.QuietOption = True
        MafftProcess.AdjustdirectionOption = True
        MafftProcess.AddOption = FileteredTrinityFasta
        MafftResult = MafftProcess.get_output()
        StrictCoverage, LargeCoverage = ApytramNeeds.calculate_coverage(MafftResult)
        logger.info("Strict Coverage: %d\tLarge Coverage: %d" %(StrictCoverage, LargeCoverage))
        print("mafft --- %s seconds ---" % (time.time() - start_mafft_time))
         
        if LargeCoverage <= LastIterStat[1]:
            Stop =True
        LastIterStat = [StrictCoverage, LargeCoverage]
        print("iteration %d --- %s seconds ---" % (i,time.time() - start_iter))
        
    
#### Write output
logger.info("End of Iterations")
logger.info("Write outputfiles")
# Best iteration
ExitCode = ApytramNeeds.write_apytram_output(FileteredTrinityFasta, ExonerateResultsDict,
                                             OutDirName+"/apytram.best.fasta", 
                                             Header = ExonerateProcess.Ryo.replace('%',"").replace("\n","").split(),
                                             Names = BestScoreNames)
# Last iteration
ExitCode = ApytramNeeds.write_apytram_output(FileteredTrinityFasta,
                                             ExonerateResultsDict,
                                             OutDirName+"/apytram.fasta",
                                             Header = ExonerateProcess.Ryo.replace('%',"").replace("\n","").split(),)

### Remove tempdir


INFO:apytram:Iterations begin
INFO:apytram:Iteration 1/10
INFO:apytram:Blast bait sequences on reads database
INFO:apytram:Retrieve sequences
INFO:apytram:Launch Trinity
INFO:apytram:Compare Trinity results with query sequences


trinity --- 0.000681161880493 seconds ---
exonerate --- 0.465357065201 seconds ---
blast --- 0.0760338306427 seconds ---

INFO:apytram:Filter sequence with a identity percentage superior to 0
INFO:apytram:Compare results with the previous iteration
INFO:apytram:Strict Coverage: 94	Large Coverage: 97



mafft --- 0.0769250392914 seconds ---

INFO:apytram:Iteration 2/10
INFO:apytram:Blast bait sequences on reads database
INFO:apytram:Retrieve sequences
INFO:apytram:Launch Trinity


...


exonerate --- 0.820907115936 seconds ---
blast --- 0.0757131576538 seconds ---

INFO:apytram:Filter sequence with a identity percentage superior to 0
INFO:apytram:Compare results with the previous iteration
INFO:apytram:Strict Coverage: 99	Large Coverage: 102



mafft --- 0.0578181743622 seconds ---

INFO:apytram:End of Iterations
INFO:apytram:Write outputfiles



iteration 3 --- 1.22465109825 seconds ---
TRINITY_DN0_c0_g1_i1
NAMES
OK
TRINITY_DN0_c0_g1_i1


In [9]:
from lib import ApytramNeeds

In [10]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 4.22582101822 seconds ---


In [11]:
import pandas
import re
df = pandas.DataFrame(ExonerateResultsDict, index = ExonerateProcess.Ryo.replace("\n","").replace('%','').split("\t"))
print df
File = open(FileteredTrinityFasta,"r")
Fasta = File.read().strip().split("\n")
File.close()
name = ""
sequence = ""
string = ""
i = 1
for line in Fasta:
    if re.match(">",line):
        name = line.split()[0].replace(">","")
        string += ">APYTRAM_%d[%s]\n" %(i,df[name]["ti"])
        i+=1
    elif name != "":
        string += line + "\n"
    else:
        pass

print string

          TRINITY_DN0_c0_g1_i1
ti   ENSMUSP00000002708_G01455
qi        TRINITY_DN0_c0_g1_i1
ql                        1339
tal                       1306
tl                        1311
tab                       1311
tae                          5
s                         5622
pi                       92.64
qab                       1315
qae                          0
>APYTRAM_1[ENSMUSP00000002708_G01455]
GCTGCTGCTAGCGAGATGTCTTCTGGTGGTCCTCGCTTCCTCGCTGCTGGTGTGCCCCGG
GCTGGCGTGCGGGCCCGGTAGGGGGTTTGGAAAGAGGCGGCACCCCAAAAAGCTGACCCC
TTTAGCCTACAAGCAGTTTATTCCCAACGTAGCCGAGAAGACCCTAGGGGCCAGCGGCAG
ATACGAAGGGAAGATCACAAGAAACTCCGAGAGATTTAAGGAACTCACCCCCAATTACAA
CCCCGACATCATATTTAAGGATGAGGAAAACACGGGAGCAGACCGACTGATGACTCAGAG
GTGCAAAGACAAGCTAAATGCCTTGGCCATCTCTGTGATGAACCAGTGGCCTGGAGTGAA
GCTACGGGTGACTGAGGGCTGGGATGAAGATGGCCATCATTCCGAGGAGTCTCTACACTA
CGAGGGCCGGGCAGTGGACATCACCACATCTGACAGGGACCGCAGCAAATATGGCATGCT
GGCCCGCCTGGCTGTGGAGGCCGGCTTCGACTGGGTCTACTATGAATCCAAAGCACACAT
CCACTGCTCTGTGAAAGCAGAGAACTCCGTGGCGGCCAAAT